In [1]:
# import dependencies

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np


# may delete or move these these once model is more defined
#  moved down to split section-> from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [2]:
# import data and create dataframe
# current path uses preliminary accident file and not real data output including vehicles
data = Path('Resources/combo_sort.csv')
df = pd.read_csv(data)
df.head(30)

,CASENUM,VEH_NO,STRATUM,VE_FORMS,NUMOCCS,NUMOCCSNAME,MONTHNAME,HARM_EV,HARM_EVNAME,MAN_COLL,...,INT_HWY,WEATHER,WKDY_IM,EVENT1_IM,MANCOL_IM,RELJCT1_IM,RELJCT2_IM,LGTCON_IM,WEATHR_IM,ALCHL_IM
0,'202002131021,3,5,4,1,1,January,12,Motor Vehicle In-Transport,6,...,0,1,7,12,6,0,2,3,1,2
1,'202002131021,4,5,4,2,2,January,12,Motor Vehicle In-Transport,6,...,0,1,7,12,6,0,2,3,1,2
2,'202002133125,1,8,1,1,1,January,42,Tree (Standing Only),0,...,0,1,4,42,0,0,1,2,1,2
3,'202002133170,1,9,1,2,2,January,42,Tree (Standing Only),0,...,0,1,4,42,0,0,3,1,1,2
4,'202002133230,1,8,2,1,1,January,12,Motor Vehicle In-Transport,1,...,0,1,1,12,1,0,3,3,1,2
5,'202002133230,2,8,2,2,2,January,12,Motor Vehicle In-Transport,1,...,0,1,1,12,1,0,3,3,1,2
6,'202002133232,1,2,1,1,1,January,9,Pedalcyclist,0,...,0,1,2,9,0,0,2,1,1,2
7,'202002133272,1,9,2,1,1,January,12,Motor Vehicle In-Transport,6,...,0,1,7,12,6,1,2,1,1,2
8,'202002133272,2,9,2,1,1,January,12,Motor Vehicle In-Transport,6,...,0,1,7,12,6,1,2,1,1,2
9,'202002133408,1,10,2,1,1,January,12,Motor Vehicle In-Transport,1,...,0,98,7,12,1,0,1,1,1,2


In [3]:
# separate targets

y = df["MAX_SEV"]
y.tail(5)


19916    1
19917    1
19918    1
19919    1
19920    0
Name: MAX_SEV, dtype: int64

In [4]:
#create resulting features dataframe
X_df = df[["VEH_NO","STRATUM","VE_FORMS","NUMOCCS","HARM_EV","MAN_COLL","UNITTYPE","HIT_RUN","MAKE","MODEL","MAK_MOD","BODY_TYP","MOD_YEAR","TOW_VEH","J_KNIFE","TRAV_SP","ROLLOVER","ROLINLOC","DEFORMED","TOWED","M_HARM","VEH_ALCH","FIRE_EXP","DR_PRES","DR_ZIP","SPEEDREL","VTRAFWAY","VNUM_LAN","VALIGN","VPROFILE","VSURCOND","VTRAFCON","VTCONT_F","P_CRASH1","P_CRASH2","P_CRASH3","PCRASH4","PCRASH5","ACC_TYPE","VPICMAKE","VPICMODEL","VPICBODYCLASS","ICFINALBODY","GVWR_FROM","GVWR_TO","BDYTYP_IM","MDLYR_IM","IMPACT1_IM","VEVENT_IM","PCRASH1_IM","NUMINJ_IM","V_ALCH_IM","REGION","URBANICITY","MONTH","YEAR","DAY_WEEK","ALCOHOL","RELJCT1","RELJCT2","TYP_INT","WRK_ZONE","REL_ROAD","LGT_COND","INT_HWY","WEATHER","WKDY_IM","EVENT1_IM","MANCOL_IM","RELJCT1_IM","RELJCT2_IM","LGTCON_IM","WEATHR_IM","ALCHL_IM"]]
X_df.tail(5)

,VEH_NO,STRATUM,VE_FORMS,NUMOCCS,HARM_EV,MAN_COLL,UNITTYPE,HIT_RUN,MAKE,MODEL,...,INT_HWY,WEATHER,WKDY_IM,EVENT1_IM,MANCOL_IM,RELJCT1_IM,RELJCT2_IM,LGTCON_IM,WEATHR_IM,ALCHL_IM
19916,1,6,2,1,12,6,1,0,12,481,...,0,1,2,12,6,0,3,3,1,2
19917,2,8,2,4,12,9,1,0,63,34,...,0,10,5,12,9,0,1,1,10,2
19918,4,6,4,1,12,1,1,0,34,34,...,0,2,4,12,1,0,3,3,2,2
19919,3,6,3,2,12,1,1,0,12,23,...,0,1,3,12,1,0,3,1,1,2
19920,1,9,2,1,12,1,1,0,49,40,...,0,1,4,12,1,0,1,1,1,2


In [5]:
# partition data into training and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, 
                                                    y, random_state =1,test_size = .2)
X_train.shape

(15936, 74)

In [6]:
#Resample the training data to account for variations in the sampe
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [7]:
# create the model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10)

In [8]:
# train the model
classifier.fit(X_resampled, y_resampled)
#classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10)

In [9]:
# make preditions with the model
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,3,3
1,1,2
2,0,0
3,0,0
4,1,1
5,3,3
6,1,1
7,1,1
8,1,1
9,0,0


In [10]:
# export results to a csv file
df_results=pd.DataFrame(results)
df_results.to_csv('results.csv')

In [11]:
# validate the model

In [12]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.8027603513174404


In [13]:
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[1817   45    7    1    0    0    7]
 [  37  694  180   20    0    0    1]
 [  14  263  297   37    0    0    0]
 [   5   36   47  338    6    0    1]
 [   0    2    8   52   20    0    0]
 [   1    3    2    0    0    1    0]
 [   9    2    0    0    0    0   32]]


In [14]:
# export confusion matrix to a csv file
df_matrix=pd.DataFrame(matrix)
df_matrix.to_csv('matrix.csv')

In [15]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1877
           1       0.66      0.74      0.70       932
           2       0.55      0.49      0.52       611
           3       0.75      0.78      0.77       433
           4       0.77      0.24      0.37        82
           5       1.00      0.14      0.25         7
           9       0.78      0.74      0.76        43

    accuracy                           0.80      3985
   macro avg       0.78      0.59      0.62      3985
weighted avg       0.80      0.80      0.80      3985



In [16]:
#